# SVM et Sélection d'attribut



## Variables d'environement

Pensez à vérifier les variables d'environement:

In [ ]:
import sys

print(sys.version)
print(sys.path)

## Séparation linéaire

Le but de cette partie est de comparer le SVM linéaire à un autre exemple de classifieur linéaire: le Perceptron. On commence d'abords par rappeler rapidement le principe du Perceptron.

### Perceptron

L'algorithm du Perceptron date des [travaux de Frank Rosenblatt](http://psycnet.apa.org/record/1959-09865-001). Le but était de modéliser l'action des neurones. Ce modèle va être ensuite utilisé pour contruire des réseaux de neurones complexes et c'est la base de toute les méthodes de Deep Learning. Le modèle donne pour chaque attribut $i \in \{1,2, \dots,d\}$ de la donnée d'entrée $x = \begin{pmatrix}x_1\\ x_2\\ \vdots \\x_d\end{pmatrix}$ un poids $w_i$. Pour chaque entrée $x$ on lui applique linéairement un vecteur de poids $w = \begin{pmatrix}w_1\\ w_2\\ \vdots \\w_d\end{pmatrix}$ pour lui attribuer un score $s = \langle w \vert x\rangle = \sum_{i=1,\dots,d}w_i.x_i$. Suite à ce score obtenu, on prends une décision:
* si $s < c \in \mathbb{R}$, on choisit la classe $0$;
* si $s \geq c $, on choisit la classe $1$

On peut écrire donc ce classifieur autrement:

$$D_{perceptron}(x) \triangleq \mathbb{1}_{\langle w \vert x\rangle + b \geq 0}$$

où $b = -c$ et $\mathbb{1}_A(x) = \begin{cases}1 & , x \in A\\0 & , x \notin A\end{cases}$.

Si on cherche à rammener les classes à la convention SVM (i.e. $y=\pm1$), avec une simple transformation affine, on a:
$$\widetilde{D}_{perceptron}(x) \triangleq 2.\mathbb{1}_{\langle w \vert x \rangle + b \geq 0} - 1 = sign(\langle w \vert x \rangle + b \geq 0)$$

### Régression logistique

Le modèle de régression logistique est proche des méthodes génératives. Ce Modèle permet juste de donner une relation entre les probabilités par classe et non pas les distributions en elle même:
$$ \ln \Big( \frac{p(x \vert y=1)}{p(x \vert y=0)}\Big) = \langle w \vert x \rangle + b$$

1. 
    a. En appliquant la règle de Bayes, montrer que: $$\ln\Big(\frac  {p(y=1\vert x)}{1-p(y=1\vert x)}\Big) = \ln\Big(\frac{p(y=1)}{p(y=0)}\Big) + b +\langle w \vert x \rangle$$
    b. Montrer donc que le décideur de la régression logistique est:
$$D_{logistic} = \sigma(\tilde b +\langle w \vert x \rangle)$$
où: $\sigma(t) \triangleq \frac{1}{1 + e^{-t}} \quad ,\forall t \in \mathbb{R}$

2. Ecrire un code python qui trace les deux fonctions, avec de multiple valeurs de $\lambda$, $t \mapsto \sigma(\lambda.t)$ et $t \mapsto \mathbb{1}_{t \geq 0}$, dans une même figure. A la lumière de la figure obtenue, discuter les deux fonctions de décisions.

#### Réponse

1. a. 

   b. 

2. 

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt


x = np.linspace(-20, 20, 1000)

lambdas = [.1, 1, 10]
colors = ['k', 'b', 'r']

# Utiliser 'plt' pour tracer les courbes qui correspondent à \sigma avec les lambda données et la fonctions Heavyside

plt.show()

### Comparaison

On rappelle ici que le SVM linéaire a pour but de maximiser la marge entre deux classes, contrairement au Perceptron et à la régression logistique. Les problèmes à optimiser ne se ressemble plus.

Le but du code, ci-dessous, est d'illustrer cette différence.

1. a. Qu'est ce que fait ce bout de code?

   b. Commentez le résultat du programme suivant.

In [ ]:
import sklearn.datasets
import sklearn.linear_model


def plot_points(points, ax, color):
    ax.scatter(points[:, 0], points[:, 1], c=color)
    

def plot_dataset(X, Y, ax, colors=['r', 'b']):
    for x, col in zip([X[Y==0], X[Y==1]], colors):
        plot_points(x, ax, col)
        

def mesh_from(instances, gap=.2):
    return np.meshgrid(
        np.arange(X[:, 0].min() - 1, X[:, 0].max() + 1, gap),
        np.arange(X[:, 1].min() - 1, X[:, 1].max() + 1, gap),
    )


def plot_contours(xx, yy, ax, classifier, **parameters):
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, **parameters)


def plot_margin(xx, yy, ax, classifier, **parameters):
    Z = np.empty(xx.shape)
    for (i, j), value in np.ndenumerate(xx):
        Z[i, j] = classifier.decision_function([[value, yy[i, j]]])[0]
    ax.contour(xx, yy, Z, [-1.0, 0.0, 1.0], colors='k', linestyles=['dashed', 'solid', 'dashed'])


X, Y = sklearn.datasets.make_blobs(n_samples=100, centers=2, random_state=0, cluster_std=0.60)
xx, yy = mesh_from(X, .01)

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
f.set_figheight(5)
f.set_figwidth(15)

for ax, loss , title in zip([ax1, ax2, ax3], ['hinge', 'perceptron', 'log'], ['SVM', 'Perceptron', 'Logistic Regression']):
    plot_dataset(X, Y, ax)
    model = sklearn.linear_model.SGDClassifier(alpha=0.01, max_iter=100, loss=loss).fit(X, Y)
    plot_contours(
        xx,
        yy,
        ax,
        model,
        cmap=plt.cm.viridis,
        alpha=0.5
    )
    plot_margin(
        xx,
        yy,
        ax,
        model
    )
    ax.set_title(title)

plt.tight_layout()
plt.show()

#### Réponse:

1. a. 

   b. 
   

### Pénalisation vs Généralisation

1. a. Entraîner des SVM linéaire avec différentes constantes de pénalisation $C$ sur les mêmes données.

   b. Tracer la marge selon les valeurs de la constante $C$.
   
   c. Commenter les résultats.

In [ ]:
import sklearn.svm

C = [.01, .1, 1, 10, 100, 1000]

f, axes = plt.subplots(1, len(C), sharey=True)
f.set_figheight(5)
f.set_figwidth(30)

for c, ax in zip(C, list(axes)):
    plot_dataset(X, Y, ax)
    # Entraîner le modèle
    # Tracer les contours et la marge
    ax.set_title('C = ' + str(c))

plt.tight_layout()
plt.show()

#### Réponse:

1. a. 

   b. 
   
   c.
   

### Kernel SVM

1. a. Relancer le même code mais cette fois-ci avec le kernel polynomial et le kernel rbf en jouant sur le $gamma$ sur les données suivantes.

   b. Commenter les résultats.
   

In [ ]:
X, Y = sklearn.datasets.make_circles(n_samples=100, factor=.3, noise=.05)
xx, yy = mesh_from(X, .01)

gammas = [.5, 1, 2, 3, 4]

f, axes = plt.subplots(2, len(gammas), sharey=True)
f.set_figheight(10)
f.set_figwidth(25)

for gamma, ax in zip(gammas, list(axes[0, :])):
    plot_dataset(X, Y, ax)
    # Entraîner le modèle
    # Tracer les contours et la marge
    ax.set_title('pol, $\gamma$ = ' + str(gamma))

gammas = [.01, .1, 2, 10, 100]
for gamma, ax in zip(gammas, list(axes[1, :])):
    plot_dataset(X, Y, ax)
    # Entraîner le modèle
    # Tracer les contours et la marge
    ax.set_title('rbf, $\gamma$ = ' + str(gamma))

plt.tight_layout()
plt.show()

#### Réponse:

1. a. 

   b. 
   

## Validation croisée

### Train-Test split

Afin d'estimer le pouvoir de généralisation d'un classifieur, il faut le tester sur de nouvelles instances. On parle de données d'entraînement ou données de tests. En pratique, on garde aussi des données de côtés pour la validation après claibrage entre entrapinements et tests.

1. a. En utilisant la fonction [train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) fournie par scikit-learn, entraîner un SVM linéaire sur 80% de vos données et tester sur le reste.

   b. Répéter l'experience plusieurs fois. Commenter les résultats

In [ ]:
import sklearn.model_selection

X, Y = sklearn.datasets.make_blobs(n_samples=100, centers=2, random_state=90, cluster_std=0.60)

X_train, Y_train = (None, None)
X_test, Y_test = (None, None)

# Répartir les données en 4/5 de train data et 1/5 de test data

C = 1
# Entraîner le modèle
classifier = None

# Tester le modèle entraîné
test_score = None

print('Test score :', test_score)

f, ax = plt.subplots(1, 1)
f.set_figheight(10)
f.set_figwidth(10)

plot_dataset(X_train, Y_train, ax)
plot_dataset(X_test, Y_test, ax, ['m', 'g'])
xx, yy = mesh_from(X, .01)
plot_margin(
    xx,
    yy,
    ax,
    model
)

plt.tight_layout()
plt.show()

### Recherche de paramètres

L'idée de la validation croisée et que l'on varie les données d'entraînement et de tests de façon à ne pas entraîner sur les mauvaises instances et puis tester sur les instances les plus durs.

On subdivise donc toutes les données en $K$ parts égales. A l'instant $k = 1,\dots,K$, on isole la $k^{ième}$ part comme ensemble de test et on entraîne notre modèle sur les $K -1$ parties restantes. On obtient donc, $K$ score d'entraînement et de test. Dans le meilleur des cas, on tombe sur les instances qui donnent le plus de pouvoir de généralisation possible.

Pour le SVM, avec juste les vecteurs supports, ce qui reprèsente moins de $10\%$ de la donnée dans notre cas, on obtient le meilleur séparateur linéaire. En cas pratique, au moment de la validation, on aura jamais vu les instances à prédire. On n'est pas sûr donc de tomber sur les vecteurs supports du meilleur modèle qui résoud le problème. On cherche donc, grâce à la validation croisée, les points les plus proches de la marge; et ainsi, le meilleur pouvoir de généralisation.

La généralisation passe aussi par le bon choix des paramètres du modèle. On utilise donc cette approche dans le but de trouver expérimentalement les meilleurs paramètres. Aussi, répète-t-on l'expérience afin d'essayer autant de configurations possibles. Les paramètres qui donnent les meilleurs scores de tests seront choisis au bout de l'étude.

Le *test score* n'est pas la seule métrique possible. On peut chercher à maximiser le *F-score*. On peut aussi s'intéresser qu'au score d'une classe donnée:

* Exemple: Vaudrait mieux un faux signal positif au scanner de bagage qu'un faux négatif (i.e. drogue ou explosif détectés comme sûrs).


1. En utilisant la focntion [cross_validate](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) de scikit-learn, trouver la bonne valeur de $C$ pour un modèle SVM linéaire.

In [ ]:
Cs = [pow(2, p) for p in range(-15, 15)]

# choisir le K de répartition
k = 5

# tester avec tout les C dans Cs est stocker les scores
test_scores = [None for C in Cs]

# le meilleur C est ?
C = None
test_score = None
print('Le meilleur paramètre de pénalisation des variables ressort est :', C, ', avec un test score de :', test_score)

# Tracer le meilleur séparateur
f, ax = plt.subplots(1, 1)
f.set_figheight(10)
f.set_figwidth(10)

plot_dataset(X, Y, ax)
xx, yy = mesh_from(X, .01)
plot_margin(
    xx,
    yy,
    ax,
    model
)

plt.tight_layout()
plt.show()

### SVM vs Random Forest

1. a. Comparer le meilleur kernel SVM trouver dans la section 'Kernel SVM' avec une forêt aléatoire de votre choix. 

   b. Tracer les courbes de séparation.

   c. Justifier votre choix de nombre d'arbres et de profondeur.

2. Commenter les résultats.

In [ ]:
import sklearn.ensemble

X, Y = sklearn.datasets.make_circles(n_samples=100, factor=.3, noise=.05)

f, ax = plt.subplots(1, 2)
f.set_figheight(10)
f.set_figwidth(20)

plot_dataset(X, Y, ax)
xx, yy = mesh_from(X, .01)
plot_margin(
    xx,
    yy,
    ax,
    model
)

plt.tight_layout()
plt.show()

## Sélection d'attribut

### Occupation des sols

L'occupation des sols à pour but de donner pour le type d'usage faits des terres. Naturellement, la manière la moins couteuse pour obtenir, à large échelle et à très grande fréquence, cette donnée, serait une approche automatique basée sur les images satellitaires.

On cherche à assigner, pour chaque pixel, un des types possibles d'usage en partant de la valeur du pixel ou son voisinage. Le problème peut être résolu avec une méthode de classification.

#### Présentation de la donnée

Pour ce TP nous utilisons une image du satellite optique [Sentinel-2 du programme européen Copernicus](http://www.esa.int/Our_Activities/Observing_the_Earth/Copernicus/Sentinel-2) acquise le 10 juillet 2016 et téléchargée depuis la plateforme [Theia](https://theia.cnes.fr).

10 des 13 bandes spectrales du satellite Sentinel-2 y sont disponibles en niveau de traitement 2A (B2, B3, B4, B5, B6, B7, B8, B8A, B11, B12). Ces 10 bandes spectrales ont été réchantillonnées en géométrie terrain (Lambert 93) à 10 m de résolution spatiale et assemblées dans le fichier *sentinel-2_sample.tif*.

L'images *sentinel-2_sample.tif* concerne une zone de $14 km\times14 km$ dans le département de la Haute-Garonne (31, ville de Saint-Gaudens).

On dispose aussi de :
* *RGE-OCS.shp* : un extrait de l’OCS GE de l’IGN sur la zone d’étude ainsi qu’un fichier décrivant;
* *RGE-foret.shp* : un extrait de la BD Forêt de l’IGN sur la zone d’étude.

A partir de ces données, on a la vérité terrain raster à la même échelle pour chaque pixel dans:
* *ground_truth_landcover.tif*: vérité terrain OCS générale.
* *ground_truth_forest.tif*: vérité terrain raster forêt-non forêt.


1. a. Ouvrir le fichier projet *dataset.qgs* avec QGIS. 

   b. Etudier l'histogramme des bandes de l'image hyperspectrale et la vérité terrain.
   
   c. Qu'est-ce-que reprèsente chaque bande spectrale de l'image?

2. a. Charger l'image sur python en se servant de *gdal*.

   b. Ajouter le *NDVI* comme bande supplémentaire à votre donnée.
    * Rappel: $$NDVI = \frac{({\mbox{NIR}}-{\mbox{Red}})}{({\mbox{NIR}}+{\mbox{Red}})}$$

   c. Séparer les pixels en données d'entraînement et données de validation à un ratio de 4/5.
   
   d. Utiliser la validation croisée pour trouver le meilleur kernel et les bons paramètres.
   
   e. Qualifier les résultats obtenus.

In [ ]:
import gdal
import gdalconst

def read(filename):
    dataset = gdal.Open(filename, gdalconst.GA_ReadOnly)
    return [dataset.GetRasterBand(band).ReadAsArray().astype(np.float) for band in dataset.RasterCount]

def add_band(X, lhs, rhs):
    # Compléter cette fonction afin qu'elle rajoute un nouveau canal à partir des deux bandes lhs et rhs
    return 

# Répondre ici

##### Commentaires:

### Sélections d'attributs:

1. a. Estimer le nombre de toutes combinaisons possibles.
   b. En utilisant les méthodes vues au cours (SVM-RFE, SFS , BFS et LR), établir une hiérarchie d'attributs (i.e. des bandes).
2. Comparer les différentes méthodes.
3. Commenter la hiérarchie obtenue.

In [ ]:
import sklearn.feature_selection

def add_best_L_attributes(X_selected, L, X, classifier):
    # Compléter la fonction
    return X_selected


def remove_worst_R_attributes(X_selected, R, X, classifier):
    # Compléter la fonction
    return X_selected


def sfs(X, classifier):
    # Compléter la fonction
    return


def bfs(X, classifier):
    # Compléter la fonction
    return

def lr(X, L, R, classifier):
    # Compléter la fonction
    return

